<a href="https://colab.research.google.com/github/nhhung1810/8-queens-SAT/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# A SAT Approach for 8 Queens Problem

## Requirement a: Formulate the problem by specifying the following points.

(Khi nào cài xong thì ghi)

## Requirement b: Write CNF clauses to describe restrictions required when Florence places a queen in the cell[3][3]

First of all, denote `b[x][y]` as "There is a queen placed at the cell (x, y)" and `-b[x][y]` as "It's invalid to place a queen at the cell (x, y)".
If Florence places a queen in the cell (3, 3) all other cells in the horizontal, vertical, main & sub diagonal line are invalid cells to place the next queen. Therefore:

$b[3][3] \implies -b[0][3] \wedge -b[1][3] \wedge -b[2][3] \wedge -b[4][3] \wedge -b[5][3] \wedge -b[6][3] \wedge -b[7][3] \wedge $

$-b[3][0] \wedge -b[3][1] \wedge -b[3][2] \wedge -b[3][4] \wedge -b[3][5] \wedge -b[3][6] \wedge -b[3][7] \wedge $

$-b[0][0] \wedge -b[1][1] \wedge -b[2][2] \wedge -b[4][4] \wedge -b[5][5] \wedge -b[6][6] \wedge -b[7][7] \wedge $

$-b[0][6] \wedge -b[1][5] \wedge -b[2][4] \wedge -b[4][2] \wedge -b[5][1] \wedge -b[6][0]$

Which equivalents to:

$-b[3][3] ∨ (-b[0][3] \wedge -b[1][3] \wedge -b[2][3] \wedge -b[4][3] \wedge -b[5][3] \wedge -b[6][3] \wedge -b[7][3] \wedge $

$-b[3][0] \wedge -b[3][1] \wedge -b[3][2] \wedge -b[3][4] \wedge -b[3][5] \wedge -b[3][6] \wedge -b[3][7] \wedge $

$-b[0][0] \wedge -b[1][1] \wedge -b[2][2] \wedge -b[4][4] \wedge -b[5][5] \wedge -b[6][6] \wedge -b[7][7] \wedge $

$-b[0][6] \wedge -b[1][5] \wedge -b[2][4] \wedge -b[4][2] \wedge -b[5][1] \wedge -b[6][0])$

Transforming into CNF clause:

$ (-b[3][3] \vee -b[0][3]) \wedge $

$ (-b[3][3] \vee -b[1][3]) \wedge $

$ (-b[3][3] \vee -b[2][3]) \wedge $

$ (-b[3][3] \vee -b[4][3]) \wedge $

$ (-b[3][3] \vee -b[5][3]) \wedge $

$ (-b[3][3] \vee -b[6][3]) \wedge $

$ (-b[3][3] \vee -b[7][3]) \wedge $

$ (-b[3][3] \vee -b[3][0]) \wedge $

$ (-b[3][3] \vee -b[3][1]) \wedge $

$ (-b[3][3] \vee -b[3][2]) \wedge $

$ (-b[3][3] \vee -b[3][4]) \wedge $

$ (-b[3][3] \vee -b[3][5]) \wedge $

$ (-b[3][3] \vee -b[3][6]) \wedge $

$ (-b[3][3] \vee -b[3][7]) \wedge $

$ (-b[3][3] \vee -b[0][0]) \wedge $

$ (-b[3][3] \vee -b[1][1]) \wedge $

$ (-b[3][3] \vee -b[2][2]) \wedge $

$ (-b[3][3] \vee -b[4][4]) \wedge $

$ (-b[3][3] \vee -b[5][5]) \wedge $

$ (-b[3][3] \vee -b[6][6]) \wedge $

$ (-b[3][3] \vee -b[7][7]) \wedge $

$ (-b[3][3] \vee -b[0][6]) \wedge $

$ (-b[3][3] \vee -b[1][5]) \wedge $

$ (-b[3][3] \vee -b[2][4]) \wedge $

$ (-b[3][3] \vee -b[4][2]) \wedge $

$ (-b[3][3] \vee -b[5][1]) \wedge $

$ (-b[3][3] \vee -b[6][0]) $

